Plot blocks per hour after Granada upgrade

In [1]:
import requests
import json
import sys

def is_fb_block(level):
    ops = requests.get(f"https://mainnet.oxheadhosted.com/chains/main/blocks/{level}/operations/3?metadata=never").json()
    simplified_ops = []
    for op_num,op in enumerate(ops):
        sop = { "contents": []}
        for c in op["contents"]:
            sc = { "fee": c["fee"],
                    "gas_limit": c["gas_limit"]}
            sop["contents"].append(sc)
        sop["fee_sum"] = sum([int(c["fee"]) for c in op["contents"]])
        sop["operation_gas"] = sum([int(c["gas_limit"]) for c in op["contents"]])
        sop["gas_ratio"] = sop["operation_gas"] / 5200000
        # operation size in bytes. We perform a binary http query and substract 68 bytes metadata
        # NOPE ! 32 (protocol hash) + 32 (hash de l'op)  + 4 (chain id) => faut enlever 68 bytes
        # 32 (hash de l'op) + 4 (chain id) + 13 (??) => faut enlever 49 bytes
        extralen=49
        sop["operation_size"] = len(bytes(requests.get(f"https://mainnet.oxheadhosted.com/chains/main/blocks/{level}/operations/3/{op_num}?metadata=never", headers={'accept': 'application/octet-stream'}).content)) - extralen
        sop["size_ratio"] = sop["operation_size"] / (512*1024)
        sop["weight"] = sop["fee_sum"] / max(sop["size_ratio"], sop["gas_ratio"])
        sop["hash"] = op["hash"][0:6]
        simplified_ops.append(sop)
#print(json.dumps(simplified_ops, indent=2))
    if (sorted(simplified_ops, key=lambda op: op["weight"], reverse = True) == simplified_ops):
        print(f"Block {level} is not a FB block")
    else:
        print("operation_hash,operation_size,operation_gas,fee_sum,weight")
        for op in simplified_ops:
            print(f"{op['hash']},{op['operation_size']},{op['operation_gas']},{op['fee_sum']},{op['weight']}")
        print(f"Block {level} is a Flashbake block!")
        exit(1)

is_fb_block(2794842)

operation_hash,operation_size,operation_gas,fee_sum,weight
op83bP,173,1495,461,1397091.1445086705
ooarpi,173,1495,461,1397091.1445086705
ooBHD1,458,3761,1172,1341627.807860262
ooQQuV,458,3761,1172,1341627.807860262
onqE5i,153,1081,372,1274739.450980392
onf6zj,178,2288,545,1238636.3636363635
ooHMDk,254,2984,691,1204155.4959785524
onmbaX,566,8203,1724,1092868.4627575278
opUJEf,409,11934,1932,841830.0653594771
oovbwS,173,5135,798,808101.2658227847
opQAUd,356,18500,2650,744864.8648648649
ooTUnn,175,9617,1275,689404.1800977436
ooHggL,248,11915,1576,687805.2874527907
ooyUqn,181,12200,1537,655114.7540983607
oog2HZ,217,15317,1887,640621.5316315205
opJKJs,453,45951,5661,640621.5316315206
onkbRg,175,28094,3120,577489.855485157
onf6TJ,175,28094,3120,577489.855485157
opXaRE,176,36067,3894,561421.7983197938
Block 2794842 is a Flashbake block!
